In [5]:
# %pip install selenium
from selenium import webdriver     
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()

In [79]:
products = []

url = 'https://www.myntra.com/the-indian-garage-co'
driver.get(url)
max_page = int(driver.find_element(By.CLASS_NAME , "pagination-paginationMeta").text.split('of')[1].strip())
for i in range(1,max_page+1):
    driver.get(url+'?p='+str(i))
    all_products = driver.find_elements(By.CLASS_NAME, "product-base")

    for product_base in all_products:
        pdp = product_base.find_element(By.TAG_NAME, "a")
        try:
            rating_cont = product_base.find_element(By.CLASS_NAME, "product-ratingsContainer")
            rating = rating_cont.find_element(By.TAG_NAME, "span")
            no_of_ratings = product_base.find_element(By.CLASS_NAME, "product-ratingsCount")
        except:
            rating = None
            no_of_ratings = None
        try:
            img = product_base.find_element(By.TAG_NAME, "img")
        except:
            img = None
        try:
            only_few_left = product_base.find_element(By.CLASS_NAME, "xcelerator-plpXceleratorInfoTag")
        except:
            only_few_left = None
        
        products.append({
            "img": img.get_attribute("src") if img else None,
            "pdp": pdp.get_attribute("href"),
            "rating": rating.text if rating else None,
            "no_of_ratings": no_of_ratings.text.split('\n')[1] if no_of_ratings else None,
            "desc": product_base.find_element(By.CLASS_NAME, "product-product").text,
            "price": ' '.join(product_base.find_element(By.CLASS_NAME, "product-price").text.split('Rs.')).strip(),
            "only_few_left": only_few_left.text if only_few_left else None
        })

In [80]:
import pandas as pd
df = pd.DataFrame(products)
df['rank'] = df.index+1
df['date']=pd.to_datetime('today').date()
df

,img,pdp,rating,no_of_ratings,desc,price,only_few_left,rank,date
0,"https://assets.myntassets.com/dpr_2,q_60,w_210...",https://www.myntra.com/jeans/the-indian-garage...,3.8,319,Men Slim Fit Jeans,519 1999(74% OFF),None,1,2023-06-08
1,"https://assets.myntassets.com/dpr_2,q_60,w_210...",https://www.myntra.com/shirts/the-indian-garag...,4.3,114,Slim Fit Casual Shirt,454 1749(74% OFF),None,2,2023-06-08
2,"https://assets.myntassets.com/dpr_2,q_60,w_210...",https://www.myntra.com/jeans/the-indian-garage...,3.7,286,Men Slim Fit Jeans,519 1999(74% OFF),None,3,2023-06-08
3,"https://assets.myntassets.com/dpr_2,q_60,w_210...",https://www.myntra.com/jackets/the-indian-gara...,None,None,Windcheater Sporty Jacket,493 1899(74% OFF),None,4,2023-06-08
4,"https://assets.myntassets.com/dpr_2,q_60,w_210...",https://www.myntra.com/jeans/the-indian-garage...,3.8,23,Men Slim Fit Jeans,519 1999(74% OFF),None,5,2023-06-08
...,...,...,...,...,...,...,...,...,...
2776,None,https://www.myntra.com/trousers/the-indian-gar...,4.4,38,Men Slim Fit Chinos Trousers,493 1899(74% OFF),Only Few Left!,2777,2023-06-08
2777,None,https://www.myntra.com/shirts/the-indian-garag...,4.2,41,Men Slim Fit Cotton Shirt,480 1849(74% OFF),Only Few Left!,2778,2023-06-08
2778,None,https://www.myntra.com/jeans/the-indian-garage...,4,153,Men Slim Fit Jeans,519 1999(74% OFF),Only Few Left!,2779,2023-06-08
2779,None,https://www.myntra.com/shirts/the-indian-garag...,4.1,178,Black Solid Casual Shirt,454 1749(74% OFF),Only Few Left!,2780,2023-06-08


In [81]:
import numpy as np
for idx, row in df.iterrows():
    driver.get(row['pdp'])
    try:
        out_of_stock_sizes = driver.find_elements(By.CLASS_NAME, "size-buttons-size-button-disabled")
        out_of_stock_sizes = [size.text for size in out_of_stock_sizes]
    except:
        out_of_stock_sizes = None
    try:
        driver.find_element(By.CLASS_NAME, 'pdp-out-of-stock')
        completely_out_of_stock = True
    except:
        completely_out_of_stock = False
    try:
        few_left_items = driver.find_elements(By.CLASS_NAME, 'size-buttons-inventory-left')
        few_left_items = [size.text for size in few_left_items if size.text != '']
    except:
        few_left_items = None
    df.at[idx, 'myntra_id'] = row['pdp'].split('/')[-2]
    df.at[idx, 'out_of_stock_sizes'] = str(out_of_stock_sizes) if out_of_stock_sizes else None
    df.at[idx, 'completely_out_of_stock'] = completely_out_of_stock
    df.at[idx, 'few_left_items'] = str(few_left_items) if few_left_items else None
    # break

KeyboardInterrupt: 

In [83]:
df.to_csv('myntra-scrape.csv', index=False)

# Image scraper from id

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/myntraid-scraper.csv')
df['Myntra Id'] = df['Myntra Id'].astype(str)

In [6]:
from selenium.webdriver.firefox.options import Options
from selenium import webdriver

options = Options()
options.add_argument("--headless")

In [7]:
for idx, row in df.iterrows():
    if idx % 100 == 0:
        driver = webdriver.Chrome(options=options) 
    driver.get(f"https://www.myntra.com/{row['Myntra Id']}")
    images = driver.find_elements(By.CLASS_NAME, "image-grid-image")
    images = [image.get_attribute('style').split('"')[1] for image in images]
    df.at[idx, 'Link'] = str(images)
    print(idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
df
# 44mins for 717 products

,Myntra Id,Link
0,19590684,"['https://assets.myntassets.com/h_720,q_90,w_5..."
1,19590682,"['https://assets.myntassets.com/h_720,q_90,w_5..."
2,19590686,"['https://assets.myntassets.com/h_720,q_90,w_5..."
3,19524370,"['https://assets.myntassets.com/h_720,q_90,w_5..."
4,19524192,"['https://assets.myntassets.com/h_720,q_90,w_5..."
...,...,...
712,7272154,"['https://assets.myntassets.com/h_720,q_90,w_5..."
713,7272191,"['https://assets.myntassets.com/h_720,q_90,w_5..."
714,7272144,"['https://assets.myntassets.com/h_720,q_90,w_5..."
715,7272170,"['https://assets.myntassets.com/h_720,q_90,w_5..."


In [9]:
df.to_csv('../data/myntraid-scraper22Jun.csv', index=False)